# Home Credit Default Risk LDA

-------------------------------------------------------------------------
        Matthieu64
--------------------------------------------------------------------------

##  Charging the datasets and librairies

In [ ]:
#import plotly 
#import plotly.plotly as py
#import plotly.graph_objs as go

import matplotlib.pyplot as plt
import seaborn as sns



#Scikit learn librairies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import scale

import pandas as pd
import numpy as np

%matplotlib inline

In [ ]:
df=pd.read_csv('../input/home-credit-default-risk/application_train.csv')
df_bureau=pd.read_csv('../input/home-credit-default-risk/bureau.csv')
dftmp= pd.read_csv('../input/home-credit-default-risk/application_train.csv')

# Exploration des données

On affiche les dimensions de la base 'application_train.csv'

In [ ]:
df.shape

On affiche les 10 premiers elements de la base de donnée.

In [ ]:
df.head(10)

Regardons le nombre de colonne de chaque type.

In [ ]:
df.dtypes.value_counts()

On affiche maintenant un graphique de la répartition de la target (le defaut de paiement).

In [ ]:
fig11=plt.figure()
ax11=plt.axes()
the_target = dftmp['TARGET']
the_target.replace(to_replace=[1,0], value= ['YES','NO'], inplace = True)
plt.title('Target repartition')
ax11 = ax11.set(xlabel='Default proportion', ylabel='Number of people')
the_target.value_counts().plot.bar()
plt.show()

La répartition de la target n'est pas équilibrée. Il y a environ 8% des individus qui ont fait défaut. Cette distribution sera à prendre en compte lors de l'analyse de nos résultats.

Ce jeu de donnée est constitué de plusieurs bases de données. Nous allons donc joindre notre base de donnée à la base de donnée 'bureau.csv'. L'ID que nous utilisons pour joindre les bases est 'SK_ID_CURR' car c'est la seule colonne commune aux deux tables.

In [ ]:
print(df.shape)
df=df.merge(right=df_bureau,how='inner', on='SK_ID_CURR')
print(df.shape)

Apres la fusion des bases, nous avons maintenant 138 colonnes et 1 465 325 lignes. Verifions qu'il n'y a pas d'invidus avec une target manquante:

In [ ]:
df.TARGET.isnull().sum()

On affiche également le nombre de colonnes par types.

In [ ]:
df.dtypes.value_counts()

On vérifie que la nouvelle distribution de la target est consistante.

In [ ]:
fig11=plt.figure()
ax11=plt.axes()
the_target = dftmp['TARGET']
the_target.replace(to_replace=[1,0], value= ['YES','NO'], inplace = True)
plt.title('Target repartition')
ax11 = ax11.set(xlabel='Default proportion', ylabel='Number of people')
the_target.value_counts().plot.pie(startangle=90, autopct='%1.1f%%')
plt.show()

Afin d'avoir une meilleure visibilité de la data nous souhaitons crées des graphiques de nos données.

Nous avons écrit une fonction afin de d'afficher la valeur de chaque individu pour chaque catégorie.

In [ ]:
#A function to print every graph with the ID as 
def print_all_values():
    df1=df.drop('SK_ID_CURR',axis=1)
    cols=df1.columns
    for col in cols:
        if (df[col].dtypes !='object'):

            fig1=plt.figure()
            ax1=plt.axes()
            plt.scatter(df[[col]],df.SK_ID_CURR,alpha=1,s=0.5)
            plt.title(col)
            ax1 = ax1.set(xlabel=col, ylabel='ID')
            plt.show()
            
            
print_all_values()

On remarque qu'il y a des valeurs aberantes, par exemple pour les salaires. Voici-ci dessous un graphique avec et sans la  valeur aberante. Ces valeurs réduisent grandement la precision des prédiction des algorithmes de machine learning, principalement la regression logistique.

In [ ]:
#Plotting the income of the people making default
df1=df[df.AMT_INCOME_TOTAL <1600000.0]
df1=df1[df.TARGET ==1 ]
df2=df[df.TARGET ==1 ]


fig2=plt.figure()
ax2=plt.axes()
plt.scatter(df2.SK_ID_CURR ,df2.AMT_INCOME_TOTAL ,alpha=1)
plt.title('Repartition des salaires sans limite de maximum')
ax2 = ax2.set(xlabel='ID', ylabel='Salaire')

fig1=plt.figure()
ax1=plt.axes()
plt.scatter(df1.SK_ID_CURR ,df1.AMT_INCOME_TOTAL ,alpha=1)
plt.title('Repartition des salaires avec une limite de maximum de 1600000$')
ax1 = ax1.set(xlabel='ID', ylabel='Salaire')
plt.show()

On peut donc supprimer cette valeur. On fait de meme pour les autres catégories où cela est le cas.

Nous ne pouvons malheuresement pas utiliser la fonction que l'on avait codé pour la précédente base de donnée car nos ordinateurs ne sont pas assez puissants pour effectuer les calculs dans un temps raisonnable. Nous nous contenterons donc d'une suppression manuelle.

In [ ]:
print(df.shape)
df=df[df.AMT_INCOME_TOTAL <1750000.0]
df=df[df.CNT_FAM_MEMBERS <12]
df=df[df.OBS_30_CNT_SOCIAL_CIRCLE <50]
df=df[df.DEF_30_CNT_SOCIAL_CIRCLE <20]
df=df[df.OBS_60_CNT_SOCIAL_CIRCLE <55]
df=df[df.DEF_60_CNT_SOCIAL_CIRCLE <15]
df=df[df.AMT_REQ_CREDIT_BUREAU_HOUR <4]
df=df[df.AMT_REQ_CREDIT_BUREAU_QRT <55]
df=df[df.CNT_CREDIT_PROLONG <6.5]
print(df.shape)

Ensuite, on remplace les valeurs manquantes lorsque cela est possible.

In [ ]:
df['OWN_CAR_AGE']=df['OWN_CAR_AGE'].fillna(0)


cols=['APARTMENTS_AVG','BASEMENTAREA_AVG','COMMONAREA_AVG','ELEVATORS_AVG','ENTRANCES_AVG','FLOORSMAX_AVG','FLOORSMIN_AVG','LANDAREA_AVG','LIVINGAPARTMENTS_AVG','LIVINGAREA_AVG','NONLIVINGAPARTMENTS_AVG','NONLIVINGAREA_AVG','APARTMENTS_MODE','BASEMENTAREA_MODE','COMMONAREA_MODE','ELEVATORS_MODE','ENTRANCES_MODE','FLOORSMAX_MODE','FLOORSMIN_MODE','LANDAREA_MODE','LIVINGAPARTMENTS_MODE','LIVINGAREA_MODE','NONLIVINGAPARTMENTS_MODE','NONLIVINGAREA_MODE','APARTMENTS_MEDI','BASEMENTAREA_MEDI','COMMONAREA_MEDI','ELEVATORS_MEDI','ENTRANCES_MEDI','FLOORSMAX_MEDI','FLOORSMIN_MEDI','LANDAREA_MEDI','LIVINGAPARTMENTS_MEDI','LIVINGAREA_MEDI','NONLIVINGAPARTMENTS_MEDI','NONLIVINGAREA_MEDI']        
for i in df.index:
    if (df.loc[i,'FLAG_OWN_REALTY'] =='N'):
        for col in cols:
            df.set_value(i,col,0)
            

df['NAME_TYPE_SUITE']=df['NAME_TYPE_SUITE'].fillna('Unknown')
df['OCCUPATION_TYPE']=df['OCCUPATION_TYPE'].fillna('Unknown')

Par exemple, pour la variable 'OWN_CAR_AGE', lorsque la personne n'a pas de voiture (le drapeau 'OWN_CAR' est à zéro), l'age de sa voiture est remplie avec 'Nan'. On remarque que la solution optimale et de remplacer les valeurs d'age manquantes par 0 (cf. graph ci-dessous).

In [ ]:
fig1=plt.figure()
ax1=plt.axes()
plt.scatter(df.OWN_CAR_AGE,df.FLAG_OWN_CAR,color='cyan')
plt.title('Graphique de "Own_Car" et "Own_Car_Age"')
ax1 = ax1.set(xlabel='Own_Car_Age', ylabel='Own_Car')
plt.show()

On en profite aussi pour remplacer la variable 'DAYS_BIRTH' par l'age de la personne pour plus de lisibilité.

In [ ]:
df['DAYS_BIRTH'] = df['DAYS_BIRTH']/(-365)
df=df.rename(columns={'DAYS_BIRTH':'AGE'})
df.AGE.describe()

Nous avons crée une fonction qui créer un dataframe avec le nom de la catégorie et le pourcentage de valeurs manquantes. 

In [ ]:
#Dataset of missing values order by percentage
def nan_count_df(df_to_print):
    
    nan_count = df_to_print.isnull().sum()

    nan_percentage = (nan_count / len(df))*100

    nan_df=pd.concat([nan_percentage], axis=1)
    nan_df=nan_df.rename(columns={0:'Percentage'})
    nan_df=nan_df[nan_df.Percentage != 0]
    nan_df = nan_df.sort_values(by='Percentage',ascending=False)
    return nan_df

nan_df=nan_count_df(df)
nan_df

Il y a des colonnes ou plus de 70% des valeurs sont manquantes. Nous ne pouvons pas remplacer ces valeurs par des moyennes ou des médiannes car elles ne seraient pas représentatives des données. Nous ne pouvons pas non plus supprimer les lignes avec des valeurs manquantes car nous supprimerions 70% des valeurs de notre base de donnée. La seule solution ici est de supprimer les colonnes avec trop de valeurs manquantes. Pour cela nous avons crée une fonction qui, à partir du dataframes précédent, supprime les colonnes avec un pourcentage de valeur manquantes supérieur à celui donné en paramètre.

In [ ]:
print(df.shape)
def delete_columns(df_transformed,df_missing_values,max_value):
        cols=df_missing_values[df_missing_values['Percentage']>=max_value].T.columns
        for col in cols:
            df_transformed=df_transformed.drop(col, axis=1)
        
        return df_transformed
df=delete_columns(df,nan_df,62)
print(df.shape)

Nous avons donc à présent 132 colonnes au lieu de 138.

# Encodage des catégories

Regardons le nombre de catégories pour chaque colonne de type 'objet'.

In [ ]:
df.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

On supprime les colonnes qui ne se sont pas avérées utiles à la prediction.

In [ ]:
#useless columns
columns_to_drop = ['SK_ID_CURR','WEEKDAY_APPR_PROCESS_START','HOUR_APPR_PROCESS_START','NAME_TYPE_SUITE','FLAG_MOBIL','FLAG_CONT_MOBILE']
df=df.drop(columns=columns_to_drop)

Maintenant, nous allons encodé les colonnes des types 'object' de deux manières différentes:

1. Pour les colonnes de 2 catégories, on les encode avec 1 et 0.
2. Pour celles qui ont plus de 2 catégories, on les encode avec la methode des OneHotEncoding pour réduire le biais induit

##### 1. Les colonnes à deux catégories

In [ ]:
#Encodage pour 2 catégories
def two_cat_encoding(df_to_transf):
    le = LabelEncoder()

    for cols in df_to_transf:
        if df_to_transf[cols].dtype == 'object':
            if len(list(df_to_transf[cols].unique())) == 2:
                le.fit(df_to_transf[cols])
                df_to_transf[cols] = le.transform(df_to_transf[cols])
    return df_to_transf
df=two_cat_encoding(df)

##### 2. Les colonnes à plus de 2 catégories

On utilise la methode get_dummies() pour faire le oneHotEncoding. Elle séparera donc les variables en utilisant des flags.

In [ ]:
df = pd.get_dummies(df)

In [ ]:
print('Les nouvelles dimensions du dataframes sont :\n', df.shape)

---------------------------------------------------------------------------------------------
#                  Algorithmes de Machine Learning
----------------------------------------------------------------------------------------------

On commence par supprimer les valeurs nulles que nous n'avons pas réussi à remplacer.

In [ ]:
df_columns=df.columns
df=df.dropna()

#### Spliting the data between training and testing

In [ ]:
X =df.drop('TARGET',axis=1)
y = df['TARGET']  

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(fit_intercept=True,intercept_scaling=1,max_iter=200,tol=0.0001,random_state=None)
logisticRegr.fit(X_train, y_train)

#### Score et erreur

In [ ]:
#ERROR
error = (1 - logisticRegr.score(X_test, y_test))*100
print('Score  = ',logisticRegr.score(X_test, y_test)*100, '%','\nErreur = %f' % error, '%')

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda=LinearDiscriminantAnalysis(n_components=None)
lda.fit(X_train, y_train)

#### Score et erreur

In [ ]:
#ERROR
error = (1 - lda.score(X_test, y_test))*100
print('Score  = ',lda.score(X_test, y_test)*100, '%','\nErreur = %f' % error, '%')

## Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=300, oob_score=True, random_state=0)
rf.fit(X_train,y_train)

#### Score et erreur

In [ ]:
error = (1 - rf.score(X_test, y_test))*100
print('Score  = ',rf.score(X_test, y_test)*100, '%','\nErreur = %f' % error, '%')

## Tree Decision Classifier

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)

#### Score et erreur

In [ ]:
error = (1 - clf.score(X_test, y_test))*100
print('Score  = ',clf.score(X_test, y_test)*100, '%','\nErreur = %f' % error, '%')

# Cross Validation

On fini par faire une cross-validation

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
predictions = logisticRegr.predict(X_test)

print(classification_report(y_test,predictions))
print('\n')
print(confusion_matrix(y_test,predictions))

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(estimator = logisticRegr , 
                         X=X_train, 
                         y=y_train, 
                         cv=3)
print('Cross-validation scores de réussite: %s' %(scores))
print('CV précision: %.3f +/- %.3f' %(np.mean(scores), np.std(scores)))

# Résultats

Après entrainement de nos algorithmes, nous avons obtenu les résultats suivants sur le jeu de test:

In [ ]:
print('Taux de réussite par modèle:\n\nRégression Logistique:',logisticRegr.score(X_test, y_test)*100,'%','\n\nLDA:',lda.score(X_test, y_test)*100,'%','\n\nRandom Forest Classifier:',rf.score(X_test, y_test)*100,'%','\n\nDecision Tree Classifier:',clf.score(X_test, y_test)*100,'%')